-----------------------------
🧱 Imports 
-----------------------------

In [ ]:
import pandas as pd
import numpy as np
import pymc as pm
import matplotlib.pyplot as plt
import arviz as az

# Load the pre-processed data

In [ ]:
df = pd.read_csv("../data/BrentOilPrices.csv")

In [ ]:
df['Date'] = pd.to_datetime(df['Date'], format='%d-%b-%y', errors='coerce')
df.set_index('Date', inplace=True)
df['Log_Return'] = np.log(df['Price']) - np.log(df['Price'].shift(1))
df.dropna(inplace=True)
log_returns = df['Log_Return'].values
n = len(log_returns)

# Define and Sample Bayesian Change Point Model

In [ ]:
with pm.Model() as model:
    # Change point
    tau = pm.DiscreteUniform('tau', lower=0, upper=n - 1)

    # Means before and after the change point
    mu_1 = pm.Normal('mu_1', mu=0, sigma=1)
    mu_2 = pm.Normal('mu_2', mu=0, sigma=1)

    # Shared standard deviation
    sigma = pm.Exponential('sigma', lam=1.0)

    # Index array
    idx = np.arange(n)

    # Piecewise mean
    mu = pm.math.switch(tau > idx, mu_1, mu_2)

    # Likelihood
    obs = pm.Normal('obs', mu=mu, sigma=sigma, observed=log_returns)

    # Sampling
    trace = pm.sample(2000, tune=1000, return_inferencedata=True)


# Trace Plots and Summary

In [ ]:
az.plot_trace(trace)
plt.show()

# Posterior of tau

In [ ]:
az.plot_posterior(trace, var_names=["tau"])
plt.title("Posterior Distribution of Change Point (tau)")
plt.show()

# Summary and Impact

In [ ]:
summary = az.summary(trace)
tau_mean = int(trace.posterior['tau'].mean().values)
date_of_change = df.index[tau_mean]

mu_1 = trace.posterior['mu_1'].mean().values
mu_2 = trace.posterior['mu_2'].mean().values
change_pct = ((mu_2 - mu_1) / abs(mu_1)) * 100

print(f"Change Point Date: {date_of_change}")
print(f"Mean Before: {mu_1:.5f}, Mean After: {mu_2:.5f}")
print(f"Approximate change: {change_pct:.2f}%")